In [1]:
import pandas as pd
import psycopg2
from config import *

ModuleNotFoundError: No module named 'config'

In [ ]:
conn = psycopg2.connect(
   database=DATABASE, user=USER, password=PASSWORD, host=HOST, port= PORT
)

In [ ]:
cursor = conn.cursor()

In [ ]:
cursor.execute("Select * FROM races LIMIT 0")
colnames = [desc[0] for desc in cursor.description]
colnames

In [ ]:
cursor.execute("SELECT * FROM races")
data = cursor.fetchall()
race_df = pd.DataFrame(data, columns=colnames)
race_df.head()

In [ ]:
race_df = race_df[race_df['year']>=1990]
race_df.shape

In [ ]:
race_df = race_df.groupby("circuitId").filter(lambda x: len(x) > 9)
race_df.shape

In [ ]:
# file_path = "../Resources/PythonExport/races_modern_high_freq.csv"
# race_df = pd.read_csv(file_path)
# race_df.head()

In [ ]:
cursor.execute("Select * FROM results LIMIT 0")
colnames = [desc[0] for desc in cursor.description]
colnames

In [ ]:
cursor.execute("SELECT * FROM results")
data = cursor.fetchall()
results_df = pd.DataFrame(data, columns=colnames)
results_df.head()

In [ ]:
# file_path = "../Resources/Dataset/results.csv"
# results_df = pd.read_csv(file_path)
# results_df.head()

In [ ]:
raceresults_df = results_df.merge(race_df,left_on='raceId',right_on='raceId',how='left')

In [ ]:
raceresults_df = raceresults_df.dropna().reset_index()

In [ ]:
raceresults_df.head()

In [ ]:
raceresults_df.drop(columns=['url','time_y','date','round','round','positionOrder','positionText','position','grid','number','points','laps','time_x','milliseconds','fastestLap','fastestLapSpeed','rank','fastestLapTime'],inplace=True)

In [ ]:
raceresults_df.head()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
km = KMeans(n_clusters=3)
y_predicted = km.fit_predict(raceresults_df[['statusId','constructorId','circuitId']])
y_predicted

In [ ]:
raceresults_df['cluster'] = y_predicted
raceresults_df

In [ ]:
import plotly.express as px


In [ ]:
fig = px.scatter_3d(
    raceresults_df,
    x="statusId",
    y="constructorId",
    z="circuitId",
    color="cluster",
    symbol="cluster",
    width=800,
#     hover_name="CoinName",
#     hover_data=["Algorithm"],
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [ ]:
import hvplot.pandas

In [ ]:
raceresults_df.hvplot.scatter(x="statusId", y="constructorId", by="cluster")


In [ ]:
import plotly.figure_factory as ff

In [ ]:
fig = ff.create_dendrogram(raceresults_df[['statusId','constructorId','circuitId']], color_threshold=10)
fig.update_layout(width=800, height=500)
fig.show()

In [ ]:
from sklearn.cluster import AgglomerativeClustering

agg = AgglomerativeClustering(n_clusters=3)
model = agg.fit(raceresults_df[['statusId','constructorId','circuitId']])

In [ ]:
raceresults_df['class']=model.labels_
raceresults_df.head()

In [ ]:
raceresults_df.hvplot.scatter(x="statusId", y="constructorId",by='class')